In [1]:
using NBInclude
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [2]:
using QuantumOptics

In [3]:
function Hofstadter_SP_Op(Nx, Ny, alpha, Basis)
    
    H_SP = Hofstadter_SP(Nx,Ny,alpha, 0)
    
    N = Nx*Ny
    
    H = SparseOperator(Basis)

    for m in 1:N
        for n in 1:N
            H = H + H_SP[m,n] * transition(Basis, m, n)
        end
    end
    
    return H
end

Hofstadter_SP_Op (generic function with 1 method)

1) Hofstadter SP QoJulia energies are has to be equal to Hofstadter Single Particle energies.

In [4]:
Nx = 4; Ny = 4; N=Nx*Ny; q = 4; N_cut = 4
PN = 2
U = 1

1

In [5]:
using LinearAlgebra

basis = NLevelBasis(N)

NLevel(N=16)

In [6]:
H1 = Hofstadter_SP_Op(Nx, Ny, 1/q, basis)

Operator(dim=16x16)
  basis: NLevel(N=16)sparse([2, 4, 5, 13, 1, 3, 6, 14, 2, 4  …  13, 15, 3, 11, 14, 16, 4, 12, 13, 15], [1, 1, 1, 1, 2, 2, 2, 2, 3, 3  …  14, 14, 15, 15, 15, 15, 16, 16, 16, 16], ComplexF64[-1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -6.0e-17 - 1.0im, -6.0e-17 + 1.0im, -1.0 + 0.0im, -1.0 + 0.0im  …  -1.0 + 0.0im, -1.0 + 0.0im, 1.0 - 1.2e-16im, 1.0 + 1.2e-16im, -1.0 + 0.0im, -1.0 + 0.0im, 1.8e-16 + 1.0im, 1.8e-16 - 1.0im, -1.0 + 0.0im, -1.0 + 0.0im], 16, 16)

In [7]:
eigenenergies(dense(H1)) == eigvals(Hofstadter_SP(Nx,Ny,1/q,0))

true

In [8]:
function get_sub_states(SP_Op, N_cut)
    
    E0, states0 = eigenstates(dense(SP_Op))
    states = states0[1:N_cut]
    
    return states
end

get_sub_states (generic function with 1 method)

In [10]:
sub_states = get_sub_states(H1, N_cut);

In [11]:
function get_Projector(states, Basis)
    
    b_sub = SubspaceBasis(Basis,states)
    P = projector(b_sub, Basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_Projector (generic function with 1 method)

In [12]:
b_sub, P, Pt = get_Projector(sub_states, basis);

In [13]:
function Subspace_Op(SP_Op, P)
    return P*SP_Op*Pt
end

Subspace_Op (generic function with 1 method)

In [14]:
H1_sub = Subspace_Op(H1, P)

Operator(dim=4x4)
  basis: Subspace(superbasis=NLevel(N=16), states:4)
 -2.82843-1.1e-16im       0.0+1.1e-16im  …   6.0e-17+8.0e-17im
 -6.0e-17-3.0e-17im  -2.82843-1.1e-16im      1.1e-16+0.0im
      0.0+0.0im       6.0e-17+1.1e-16im          0.0+0.0im
  6.0e-17+0.0im       1.1e-16+0.0im         -2.82843-1.0e-17im

In [16]:
num_sub_list = []
for m in 1:N
    NM = transition(basis, m, m)
    NMP = Subspace_Op(NM, P)
    push!(num_sub_list, NMP)
end

In [17]:
states_mb = bosonstates(b_sub, PN) 
basis_mb = ManyBodyBasis(b_sub, states_mb)

ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=16), states:4), states:10)

In [18]:
function get_MB_Op(MB_Basis, Basis, Op)
    
    Op_MB = SparseOperator(MB_Basis)
    for i in 1:length(Basis)
        for j in 1:length(Basis)
            Op_MB = Op_MB + Op.data[i,j] * transition(MB_Basis, i, j)
        end
    end
    
    return Op_MB
end

H1_MB = get_MB_Op(basis_mb, b_sub, H1_sub)

Operator(dim=10x10)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=16), states:4), states:10)sparse([1, 2, 4, 1, 2, 3, 4, 5, 7, 1  …  9, 3, 4, 6, 7, 9, 10, 4, 7, 10], [1, 1, 1, 2, 2, 2, 2, 2, 2, 3  …  8, 9, 9, 9, 9, 9, 9, 10, 10, 10], ComplexF64[-5.656854249492381 - 2.2e-16im, -8.0e-17 - 4.0e-17im, 8.0e-17 + 0.0im, 0.0 + 1.6e-16im, -5.65685424949238 - 2.2e-16im, 6.0e-17 + 1.1e-16im, 1.1e-16 + 0.0im, -8.0e-17 - 4.0e-17im, 6.0e-17 + 0.0im, -1.6e-16 - 1.6e-16im  …  8.0e-17 - 2.0e-17im, 6.0e-17 + 8.0e-17im, -1.1e-16 - 1.1e-16im, 1.1e-16 + 0.0im, 1.0e-16 - 1.1e-16im, -5.6568542494923815 - 1.2e-16im, 8.0e-17 - 2.0e-17im, 8.0e-17 + 1.2e-16im, 1.6e-16 + 0.0im, -5.6568542494923815 - 2.0e-17im], 10, 10)

In [52]:
# H1_mb = manybodyoperator(basis_mb, H1_sub)

In [19]:
num_mb_list = []
for m in 1:N
    NMP = get_MB_Op(basis_mb, b_sub, num_sub_list[m])
    push!(num_mb_list, NMP)
end

In [23]:
U = 2

function Int_Op(MB_Num_Op_List, MB_Basis, U)

    IT = SparseOperator(MB_Basis)
    
    for m in 1:N
        IT = IT + U/2 * ( MB_Num_Op_List[m]*MB_Num_Op_List[m] - MB_Num_Op_List[m] )
    end
    
    return IT
end

H_Int_MB = Int_Op(num_mb_list, basis_mb, U)

Operator(dim=10x10)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=16), states:4), states:10)sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  10, 10, 10, 10, 10, 10, 10, 10, 10, 10], ComplexF64[-1.28936050169679 + 0.0im, 0.00757962569811259 + 0.01185689701910187im, -0.0437258336476612 + 0.02661961296988589im, -0.01095277178701358 + 0.04650569240555596im, -0.01893974682379918 + 0.06663927799719217im, 0.02022746845292802 + 0.01717055615191673im, -0.00575227287204036 + 0.04613495742640633im, 0.03793323862671036 - 0.01164801203674734im, 0.02858758182358304 - 0.05907386795967209im, 0.01184660099881899 + 0.012708215570994im  …  0.01184660099881899 - 0.012708215570994im, 0.0161342208377085 - 0.02433646597608097im, -0.03492065003154119 + 0.00192034448810921im, 0.00262592956589625 + 0.00030173629125369im, 0.0359038153203539 - 0.01402817893925641im, -0.03498222498650828 + 0.01584048041542828im, -0.00846315454922784 - 0.00157

In [24]:
basis2 = basis ⊗ basis

# interaction : at_i at_i a_i a_i = at_i a_i at_i a_i - at_i a_i = n_i n_i - n_i
    
Vint2 = SparseOperator(basis2)

for n in 1:N
    Vint2 += U/2*transition(basis,n,n)⊗transition(basis,n,n)
end

Vint2_sub = (P⊗P)*Vint2*(Pt⊗Pt)

Vint_mb = manybodyoperator(basis_mb, Vint2_sub)

Operator(dim=10x10)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=16), states:4), states:10)
    0.210639+0.0im        …    0.0118466-0.0127082im
  0.00757963+0.0118569im       0.0161342-0.0243365im
  -0.0437258+0.0266196im      -0.0349207+0.00192034im
  -0.0109528+0.0465057im      0.00262593+0.000301736im
  -0.0189397+0.0666393im       0.0359038-0.0140282im
   0.0202275+0.0171706im  …   -0.0349822+0.0158405im
 -0.00575227+0.046135im      -0.00846315-0.00157627im
   0.0379332-0.011648im        0.0268504+0.0243677im
   0.0285876-0.0590739im      -0.0044525-0.00459885im
   0.0118466+0.0127082im        0.273437+0.0im

In [25]:
H_MB = H1_MB + H_Int_MB
H_MB = H1_MB + Vint_mb

Operator(dim=10x10)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=16), states:4), states:10)
    -5.44621-2.2e-16im    …    0.0118466-0.0127082im
  0.00757963+0.0118569im       0.0161342-0.0243365im
  -0.0437258+0.0266196im      -0.0349207+0.00192034im
  -0.0109528+0.0465057im      0.00262593+0.000301736im
  -0.0189397+0.0666393im       0.0359038-0.0140282im
   0.0202275+0.0171706im  …   -0.0349822+0.0158405im
 -0.00575227+0.046135im      -0.00846315-0.00157627im
   0.0379332-0.011648im        0.0268504+0.0243677im
   0.0285876-0.0590739im      -0.0044525-0.00459885im
   0.0118466+0.0127082im        -5.38342-2.0e-17im

2) If U<<Band-Gap, Sub-Space Hofstadter Finite-U energies converges at Hofstadter Finite-U energies.
3) When all SP states projected to the new sub-space, Sub-Space Hofstadter Finite-U energies have to equal to Hofstadter Finite-U energies.

In [26]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")

Hofstadter_Finite_U (generic function with 1 method)

In [31]:
# using Plots

E1 = eigenenergies(dense((H_MB+dagger(H_MB))/2))

E2 = eigenenergies(dense(Hofstadter_Finite_U(Nx, Ny, 1/q, PN, U)))

# print(E1-E2[1:length(E1)])

# plot(1:length(E1),E1,seriestype=:scatter,markershape=:star5, markersize=6, label="Sub-Space")
# plot!(1:length(E2),E2,seriestype=:scatter,label="Real-Space",legend=:topleft)
# xlabel!("n");ylabel!("E")

136-element Vector{Float64}:
 -5.649088257988248
 -5.649088257988246
 -5.566797429509227
 -5.566797429509225
 -5.473445191751442
 -5.473445191751442
 -5.47344519175144
 -5.473445191751436
 -5.377714259598001
 -5.377714259597988
 -2.8284271247461934
 -2.8284271247461925
 -2.828427124746192
  ⋮
  3.9999999999999942
  4.000000000000008
  5.674115940328522
  5.6741159403285275
  5.852756589683972
  5.852756589683979
  6.0231091360459414
  6.023109136045951
  6.023109136045961
  6.023109136045963
  6.148762358737362
  6.148762358737363

In [32]:
E1

10-element Vector{Float64}:
 -5.646130944789018
 -5.646130944789016
 -5.5318542494923815
 -5.531854249492378
 -5.406854249492384
 -5.406854249492384
 -5.406854249492382
 -5.4068542494923815
 -5.292577554195746
 -5.292577554195745